# Amazon DynamoDB

<img src="../_assets/aws_service_icons/dynamodb.svg" width="80" alt="Amazon DynamoDB">

## Goals
- Understand what DynamoDB is
- Know what it’s practically used for
- See a simple **SDK pseudo-code** example (not executed)


## What is DynamoDB?

**Amazon DynamoDB** is a fully managed **NoSQL** database service on AWS.

- Data is stored in **tables** made of **items** (rows) with **attributes** (columns).
- It’s optimized for **low-latency** access (typically single-digit milliseconds) and can scale to very high throughput.
- It supports **key-value** and **document** access patterns.

In practice, DynamoDB is designed around **access patterns** you know upfront, using a **primary key** (and optionally indexes) to retrieve data efficiently.


## What is DynamoDB used for (practically)?

DynamoDB is commonly used when you need **fast, predictable reads/writes**, elastic scaling, and don’t need relational joins.

Typical use cases:
- **User profiles**, preferences, and metadata
- **Session stores** and authentication/session tokens
- **Shopping carts** and order state (state machines)
- **IoT/device telemetry** (time-ordered with sort keys)
- **Event logs** and idempotency keys
- **Application feature flags** and configuration

Common architectural fit:
- Serverless / microservice backends where each service owns a table
- Workloads with **spiky traffic** (choose on-demand or auto-scaling)

When it’s a poor fit:
- Heavy **ad-hoc analytics** or full-table scans as a primary workflow
- Complex relational queries/joins (consider RDS/Postgres)
- Search and text relevance (consider OpenSearch)


## Core concepts (minimum you should know)

- **Primary key**
  - **Partition key**: determines where an item lives (distribution/scaling)
  - **Sort key** (optional): enables multiple items per partition key and range queries
- **Secondary indexes**
  - **GSI** (Global Secondary Index): alternate query pattern (different key)
  - **LSI** (Local Secondary Index): alternate sort key within same partition key
- **Consistency**: reads can be **eventually consistent** or **strongly consistent** (API-dependent)
- **Capacity**
  - **On-demand**: pay per request, great for spiky/unknown workloads
  - **Provisioned**: fixed capacity (with optional auto-scaling)
- **Conditional writes**: enforce invariants without locks (e.g., “only create if absent”)
- **Transactions**: multi-item, ACID-style operations (higher cost/latency)


## Using DynamoDB with an SDK (pseudo-code)

This is **illustrative pseudo-code** showing the typical flow in an AWS SDK (e.g., Python `boto3`, JS v3, Java, etc.).

**Do not run as-is**: fill in real credentials/auth, region, and error handling.

```python
# PSEUDO-CODE (not executed)

# 1) Create a DynamoDB client/resource
sdk = AwsSdk()
ddb = sdk.dynamodb(region="us-east-1")

table_name = "Users"

# 2) Create a table (one-time setup)
# Primary key: (PK) user_id
ddb.create_table(
    TableName=table_name,
    KeySchema=[{"AttributeName": "user_id", "KeyType": "HASH"}],
    AttributeDefinitions=[{"AttributeName": "user_id", "AttributeType": "S"}],
    BillingMode="PAY_PER_REQUEST",  # or ProvisionedThroughput=...
)

# 3) Put an item
ddb.put_item(
    TableName=table_name,
    Item={
        "user_id": {"S": "u_123"},
        "email": {"S": "user@example.com"},
        "created_at": {"S": "2026-01-06T00:00:00Z"},
    },
)

# 4) Get an item by primary key
resp = ddb.get_item(
    TableName=table_name,
    Key={"user_id": {"S": "u_123"}},
    ConsistentRead=True,  # optional
)
user_item = resp.get("Item")

# 5) Update an item (atomic)
ddb.update_item(
    TableName=table_name,
    Key={"user_id": {"S": "u_123"}},
    UpdateExpression="SET last_login = :ts",
    ExpressionAttributeValues={":ts": {"S": "2026-01-06T12:34:56Z"}},
)

# 6) Conditional write ("create only if not exists")
ddb.put_item(
    TableName=table_name,
    Item={"user_id": {"S": "u_999"}, "email": {"S": "new@example.com"}},
    ConditionExpression="attribute_not_exists(user_id)",
)

# 7) Delete an item
ddb.delete_item(TableName=table_name, Key={"user_id": {"S": "u_123"}})
```


## Pitfalls & quick tips

- Avoid **hot partitions**: design partition keys to spread traffic.
- Model around **queries you need**; add GSIs for alternate access patterns.
- Be careful with **scans** (expensive) vs **queries** (key-based).
- Prefer **conditional writes** to prevent race conditions.

## References
- AWS Docs: DynamoDB (concepts, best practices, SDK examples)
- AWS Docs: DynamoDB Data Modeling (single-table design patterns)
